# Progetto AADS 
## Stima del numero di nodi esterni (foglie) dato in numero di nodi interni

#### Autore: Lorenzo Pratesi Mariti - 7037171 - <a href="mailto:lorenzo.pratesi@stud.unifi.it">lorenzo.pratesi@stud.unifi.it</a>

##### Descrizione del progetto:
In questo notebook verranno effettuate verifiche numeriche sul numero medio di foglie nelle strutture dati ad albero. In partcolare vedremo in dettaglio le strutture dati come:
- alberi $s$-ari;
- alberi binari;
- alberi planari con radice;
- alberi binari di ricerca.

### Preparazione:
Per prima cosa è necessario importare tutti i pacchetti necessari allo svolgimento di questo progetto:

In [1]:
import random 
from tqdm import tqdm
from sage.combinat.species.library import *
#from sympy import init_printing
#init_printing()

In [2]:
from lib.sage_itertools import *
from lib.backtraking import *
from lib.mary_tree import *

## Alberi $s$-ari
Un albero $s$-ario è una struttura definita in modo ricorsivo come un nodo esterno oppure un nodo interno che è connesso ad $s$ alberi $s$-ari. Se si indica con $\mathcal{T}$ la classe degli alberi $s$-ari, questa definizione si traduce nella seguente relazione:

$$\mathcal{T} = \{\bullet\} + \{\circ\} × \mathcal{T} × \mathcal{T} · · · × \mathcal{T}$$

Applicando il metodo simbolico, si ha che la funzione generatrice che conta il numero di alberi $s$-ari con $n$ nodi interni soddisfa l'equazione: ${T}(t) = 1 + {T}(t)^s$
Per l’estrazione del coefficiente si procede come segue:

$$T(t) − 1 = tT(t)^s,\quad T_0 = 1$$

$$A(t) = t(1 + A(t))^s,\quad A(t) = T(t) − 1$$

$$T_n = A_n =\frac{1}{n}[u^{n−1}](1 + u)^{sn} =\frac{1}{n}\binom{sn}{n − 1}=\frac{1}{(s − 1)n + 1}\binom{sn}{n}.$$

### Verifica del risultato del numero di alberi $s$-ari dato il numero di nodi interni.
Definiamo l'arietà di ogni nodo

In [3]:
# 5-ary trees
s = 5
n = 5

La struttura ricorsiva per un albero $s$-ario è la seguente: 
$$T= L + N \times \underbrace{T \times T \times \cdots \times T}_{s} = L+N\times T^s$$
- $L$: un nodo esterno, rappresentato dalla classe combinatoriale vuota `EmptySetSpecies`
- $N$: nodo interno a cui sono collegati $s$ alberi $s$-ari, rappresentato dalla classe combinatoriale singleton `SingletonSpecies`
- $T$: struttura ricorsiva definita dalla classe combinatoriale `CombinatorialSpecies`

In [4]:
o = var('o')
L = EmptySetSpecies()
N = SingletonSpecies() 
TreeS = CombinatorialSpecies()

TreeS.define(L + N * TreeS ^ s)
Tree5 = TreeS.isotypes([o] * n).cardinality()

Stampiamo la funzione generatrice e controlliamo l'uguaglianza al valore teorico $\frac{1}{(s − 1)n + 1}\binom{sn}{n}.$

In [11]:
show('Generating Function:')
show(TreeS.isotype_generating_series())

show('Generating Function Coefficients:')
print(TreeS.isotype_generating_series().coefficients(n))

'Generating Function:'

1 + x + 5*x^2 + 35*x^3 + 285*x^4 + 2530*x^5 + O(x^6)

'Generating Function Coefficients:'

[1, 1, 5, 35, 285]


In [12]:
def val_teorico(s, n):
    return 1 / ((s - 1) * n + 1) * binomial(s * n, n)

[val_teorico(s, n) for n in range(n)]

[1, 1, 5, 35, 285]

Le due serie ottenute coincidono.

La definizione ricorsiva sopra esposta è otenibile ugualmente tramite l'utilizzo della classe `MAryTrees`. 
Testiamo ora la definizione di albero $s$-ario, tramite questa classe e definiamo la funzione `get_random_of` che estrae una permutazione della struttura in modo uniformemente casuale;

In [15]:
s = 5
n = 5
tree_s = MAryTrees(s)
tree_s

5-ary trees

Ricordando che il numero di strutture possibili è dato da $k = \frac{1}{(s − 1)n + 1}\binom{sn}{n}$ possiamo generare un numero casuale $r$ nell'intervallo $(0, k-1)$ ed estrarre la struttura di indice $r$. Questa è un implementazione a forza bruta. La sua complessità è lineare $O(r)$, dove $r$ è la cardinalità dell'insieme in ingresso.

In [18]:
def get_random_of(trees):
    rand_index = randint(0, trees.cardinality() - 1)
    for idx, tree in enumerate(trees):
        if idx == rand_index:
            return tree
    return None

random_s_tree = get_random_of(tree_s._get_m_ary_trees_size(n))
random_s_tree

[[., ., ., ., .], [., ., ., ., .], [., [., ., ., ., .], ., ., .], ., .]

### Calcoliamo il numero totale di strutture possibili

Costruiamo un albero di arietà 5 e restituiiamo l'insieme degli alberi $s$-ari di dimensione $n = 10$. Dopodichè calcoliamo la dimensione dell'insieme.

In [7]:
concrete_count = len(tree_s._get_m_ary_trees_size(10))
concrete_count

Calcoliamo il valore teorico tramite la formula proposta dalla teoria
$$\frac{1}{(s-1) \cdot n+1}\binom{s\cdot n}{n}$$

Calcolo tramite la funzione del pacchetto MaryTree

In [8]:
theoretical_count = tree_s._get_m_ary_trees_size(n).cardinality()
theoretical_count

2530

Calcolo esplicitando la formulazione teorica

In [9]:
def val_teorico(s, n):
    return 1 / ((s - 1) * n + 1) * binomial(s * n, n)

val_teorico(s, n)

2530

Come si evince i due risultati coincidono.

Proponiamo quindi anche una procedura che, dato un valore $k$ (pari al numero di simulazioni da condurre, ovvero il numero di alberi da creare), restituisce il numero totale di foglie presenti all'interno di tutti i $k$ alberi $s$-ari creati

In [29]:
from functools import lru_cache

def count_leaves(tree, leave_structure="[., .]"):
    return ' '.join(map(str, tree)).count(leave_structure)

def get_leaves_str(size):
    structure = '['
    for _ in range(size - 1):
        structure+='., '
    
    return structure + '.]'

@cached_function
def count_leaves_s_tree(s, n, k):
    acc = 0
    leave_struct = get_leaves_str(s)
    tree_s = MAryTrees_size(s, n)
    
    for _ in tqdm(range(1, k + 1)):
        tree = get_random_of(tree_s)
        cl = count_leaves(tree, leave_structure=leave_struct)
        acc += cl
        #print(f"internal nodes: {n - cl}, external nodes: {cl}")

    return acc

Richiamiamo la procedura precedente con $k=1000$ (ovvero con $1000$ simulazioni di alberi $s$-ari) e salviamo il risultato all'interno di una variabile:

In [30]:
ary = 5
num_of_node = 5
iteration = 1000
leaves = count_leaves_s_tree(ary, num_of_node, iteration)
leaves

100%|██████████| 1000/1000 [12:03<00:00,  1.38it/s]


1884

## 2. Alberi Binari
Vogliamo determinare il numero medio di foglie in un albero binario con n nodi interni. Una foglia è un nodo interno in cui entrambi i figli sono nodi esterni. Si può usare il metodo simbolico:

$$\mathcal{B}=\{\bullet\}+\{\circ\}\times \mathcal{B}\times \mathcal{B}$$

Sia $B(t, w)$ la funzione generatrice bivariata che conta il numero $b_{n,k}$ di alberi binari con $n$ nodi interni e $k$ foglie, allora:

$$B(t, w) = 1 + tw + tB(t, w)^2 − t$$

(si sottrae $t$ per evitare che l’albero di misura 1 venga contato 2 volte). Derivando rispetto a $w$ e poi ponendo $w = 1$ si ha: 

$$B_w(t, 1) = t + 2tB(t, 1)B_u(t, 1) = \frac{t}{1 − 2tB(t, 1)}=\frac{t}{\sqrt{1 − 4t}}$$

Quindi, il numero medio di foglie in un albero binario con n nodi interni è:

$$\frac{[t^n]\frac{1}{\sqrt{1−4t}}}{\frac{1}{n+1}\binom{2n}{n}} = \frac{\binom{2n−2}{n−1}}{\frac{1}{n+1}\binom{2n}{n}}=\frac{n(n + 1)}{2(2n − 1)} \approx \frac{n}{4} \quad{\text{ per }} n\rightarrow \infty$$

###  Verifica del numero di foglie negli alberi binari dato il numero di nodi interni.  
Numero di nodi esterni negli alberi binari: $$ \frac{n}{4} \quad \text{per}\quad n \rightarrow \infty.$$


Definiamo la struttura dei suddetti alberi

La struttura ricorsiva per un albero binario è la seguente: 
$$T = L + N \times T \times T $$
- $L$: un nodo esterno, rappresentato dalla classe combinatoriale vuota `EmptySetSpecies`
- $N$: nodo interno a cui sono collegati due alberi binari, rappresentato dalla classe combinatoriale singleton `SingletonSpecies`
- $T$: struttura ricorsiva definita dalla classe combinatoriale `CombinatorialSpecies`

In [36]:
n = 10
o = var('o')
L = EmptySetSpecies()
N = SingletonSpecies()
BT = CombinatorialSpecies()

BT.define(L + N * BT * BT)
BTn = BT.isotypes([o] * n).cardinality()

Stampiamo la funzione generatrice e controlliamo l'uguaglianza al valore teorico dei numeri di Catalan $\frac{1}{n + 1}\binom{2n}{n}.$ 

In [37]:
show('Generating Function:')
show(BT.isotype_generating_series())

show('Generating Function Coefficients:')
print(BT.isotype_generating_series().coefficients(n))

'Generating Function:'

1 + x + 2*x^2 + 5*x^3 + 14*x^4 + 42*x^5 + 132*x^6 + 429*x^7 + 1430*x^8 + 4862*x^9 + 16796*x^10 + O(x^11)

'Generating Function Coefficients:'

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]


In [38]:
[catalan_number(i) for i in range(n)]

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]

Le due serie coincidono.

Lo stesso risultato è ottenibile con la classe `BinaryTrees`

In [58]:
n = 100
BT = BinaryTrees(n)
BT

Binary trees of size 100

Esempio di struttura ad albero Binario contenente 10 nodi

In [54]:
ascii_art(get_random_of(BinaryTrees(10)))

  _____o_____
 /           \
o             o
             /
           _o__
          /    \
        _o_     o
       /   \    
      o     o   
     / \        
    o   o       

Definiamo ora una procedura che, dati due parametri $k$ e $n$, tramite simulazione ritorna il valore totale delle foglie presenti all'interno di $k$ alberi binari con $n$ nodi interni

In [68]:
def count_leaves_bin(t, k, n):
    acc = 0
    for _ in tqdm(range(k)):
        tree = t.random_element()
        acc += count_leaves(tree)
        
    return acc

Ora richiamiamo la procedura con $k = 10000$ numero di alberi e $n = 100$ numero di nodi interni di ogni albero

In [79]:
k = 10000
n = 100
BT = BinaryTrees(n)
tot_leaves = count_leaves_bin(BT, k, n)
tot_leaves

100%|██████████| 10000/10000 [00:13<00:00, 767.21it/s]


254214

Ora prendiamo il valore ritornato, lo dividiamo per il numero di alberi su cui abbiamo effettuato la simulazione, e notiamo che il risultato tende a $\frac{n}{4}$

In [80]:
(tot_leaves / k).n()

25.4214000000000

In questo caso $n$ era uguale a $100$, quindi correttamente si ottiene un valore attorno a $25$.

In [81]:
n / 4

25

## 3. Alberi planari con radice
Sia $G(z, u)$ la funzione generatrice bivariata associata a $G_{n,k}$, ovvero:

$$G(z,u)=\sum_{n,k\ge 0}G_{n,k}z^nu^k$$

dove $z$ conta il numero di nodi e $u $quello delle foglie. La classe $\mathbb{A}$ degli alberi aventi nodi di qualsiasi arietà $s$ risulta essere l’unione disgiunta degli insiemi $\mathbb{A}_s$, $s \ge 0$, costituiti dagli alberi di $\mathbb{A}$ che hanno un nodo di arietà $s$ come radice:

$$\mathbb{A} = \{\circ\} \cup \{\circ\} \times \mathbb{A} \cup \{\circ\} \times \mathbb{A} \times \mathbb{A} \cup \{\circ\} \times \mathbb{A} \times \mathbb{A} \times \mathbb{A} \cup \cdots$$

Passando alla funzione generatrice bivariata si ha:

$$G(z,u)=zu+zG(z,u)+zG(z,u)^2+\cdots=zu+z\sum_{k\ge 0}G(z,u)^k=zu+\frac{zG(z,u)}{1-G(z,u)}=z\left(u+\frac{G(z,u)}{1-G(z,u)}\right)$$

Per trovare $G_{n,k}$ si applica la formula di inversione di Lagrange:

$$G_{n,k} = [u^k][z^n]G(z, u) = [u^k]\frac{1}{n}[y^{n-1}]\left(u +\frac{y}{1 - y}\right)^n=\frac{1}{n}\binom{n}{k}[y^{n-1}]\frac{y^{n-k}}{(1 - y)^{n-k}}=\frac{1}{n}\binom{n}{k}\binom{n - 2}{k - 1}.$$

Avendo trovato $G_{n,k}$ in forma esplicita, il numero medio di foglie questa volta può essere determinato calcolando le sequenti somme combinatorie

$$\frac{\sum_{k\ge 0}kG_{n,k}}{\sum_{k\ge 0}G_{n,k}}=\frac{\frac{1}{2}\binom{2n-2}{n-1}}{\frac{1}{n}\binom{2n-2}{n-1}}=\frac{n}{2}$$



Definiamo la struttura dei suddetti alberi

La struttura ricorsiva per un albero planare con radice è la seguente: 
$$T = N + (N \times T) + (N \times T \times T) + (N \times T \times T \times T) + \cdots = N + \sum_{j=1}^{n-1} (N \times T^j)$$
- $N$: nodo interno, rappresentato dalla classe combinatoriale singleton `SingletonSpecies`
- $T$: struttura ricorsiva definita dalla classe combinatoriale `CombinatorialSpecies`

In [124]:
n = 10
o = var('o')
N = SingletonSpecies()
TreeP = CombinatorialSpecies()

tree_def = N  

for j in range(1, n + 1):
    tree_def += N * TreeP^j

TreeP.define(tree_def)
TreeP.isotypes([o] * n).cardinality()

4862

Il numero di alberi planari con radice, classificati in base al numero di foglie, equivalgono ai numeri di Catalan shiftati di una posizione, ovvero equivalgono al $(n-1)$-esimo numero di Catalan.

In [126]:
show('Generating Function:')
show(TreeP.isotype_generating_series())

show('Generating Function Coefficients:')
print(TreeP.isotype_generating_series().coefficients(n + 1))

'Generating Function:'

x + x^2 + 2*x^3 + 5*x^4 + 14*x^5 + 42*x^6 + 132*x^7 + 429*x^8 + 1430*x^9 + 4862*x^10 + O(x^11)

'Generating Function Coefficients:'

[0, 1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]


In [136]:
[catalan_number(i - 1) for i in range(n + 1) if i > 0]

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]

### Verifica del numero di foglie negli alberi planari dato il numero di nodi interni.

Il numero di nodi foglia degli alberi planari tende a $\frac{n}{2}$.

La definizione ricorsiva sopra esposta è otenibile ugualmente tramite l'utilizzo della classe `OrderedTrees`.

In [137]:
OrderedTrees(10).cardinality()

4862

In [140]:
ascii_art(OrderedTrees(10).random_element())

  __o___
 / /   / 
o o   o
  |   |
  o   o
      |
      o
      |
      o_
     / /
    o o

Definiamo ora una procedura che, dati due parametri $k$ e $n$, tramite simulazione ritorna il valore totale delle foglie presenti all'interno di $k$ alberi planari con n nodi interni

In [141]:
def count_leaves_planar_tree(t, k, n):
    acc = 0
    for i in tqdm(range(k)):
        tree = t.random_element()
        acc += n - count_leaves(tree, leave_structure="[]")
    
    return acc

Ora richiamiamo la procedura con i valori di $k=10000$ (ovvero $10000$ alberi) e $n=100$ (alberi con 100 nodi interni ciascuno) e salviamo il risultato del totale delle foglie  all'interno di una variabile:

In [143]:
k = 10000
n = 100
OT = OrderedTrees(n)
tot_leaves_plan = count_leaves_planar_tree(OT, k, n);
tot_leaves_plan

100%|██████████| 10000/10000 [00:07<00:00, 1382.04it/s]


500372

In [144]:
(tot_leaves_plan / k).n()

50.0372000000000

In questo caso $n =100$, e quindi il risultato ottenuto di circa $50$ nodi foglia rispecchia il risultato teorico di $\frac{n}{2}$.

In [145]:
n / 2

50

## 4. Alberi binari di ricerca
Un albero binario di ricerca è un albero binario con chiavi associate a ciascun nodo interno con la proprietà che la chiave in un nodo è maggiore di tutte le chiavi nel suo sottoalbero sinistro e minore di tutte quelle nel sottoalbero destro.

- Il numero medio di confronti fra chiavi necessari per costruire un albero binario di ricerca inserendo $n$ chiavi distinte in ordine casuale è pari alla lunghezza media del cammino totale interno 
$$I_n = n − 1 + \frac{1}{n}\sum^n_{k=1}(I_{k−1} + I_{n−k}),\quad I_0 = 0. \qquad \text{
Si trova} \quad I_n = 2(n + 1)(H_{n+1} − 1) − 2$$

-  Numero medio di confronti in una ricerca con successo: 
$$\frac{1}{n}I_n + 1 = 2H_n − 3 − 2\frac{H_n}{n}$$

-  Numero medio di confronti in una ricerca senza successo: $$\frac{I_n + 2n}{n + 1} = 2H_{n+1} − 2 $$

### Numero medio di foglie negli alberi binari di ricerca dato il numero di nodi interni. 
Il numero medio di foglie in un albero binario di ricerca costruito inserendo $n$ chiavi distinte in ordine casuale soddisfa la ricorrenza:

$$F_n = \delta_{n,1} + \frac{2}{n}\sum^{n−1}_{k=0}F_k,\quad F_0 = 0$$
$\delta_{n,1}$ è la porzione di costo associata alla radice). Passando alle funzione generatrice si trova:

$$tF'(t) = t +\frac{2t}{1 − t}F(t)$$

$$F(t) = \frac{1}{3}\frac{1}{(1 − t)^2}+\frac{1}{3}(t-1)$$
Estraendo il coefficiente si trova infine:
$$F_n =\frac{n + 1}{3}.$$

Il numero medio di foglie all'interno degli alberi binari di ricerca tende a $\frac{n}{3}$.

Siccome gli alberi binari di ricerca non possiedono una struttura definita a priori e regolare come nel caso degli alberi binari standard, o degli alberi $s$, definiamo una procedura che, data una permutazione di $m$ elementi, restituisce una struttura ad albero corrispondente all'albero binario di ricerca costruito su quella specifica permutazione.  Ciò ci permette di calcolare, analogamente a quanto effettuato per le altre analisi, il numero medio di foglie e di confrontarlo con il risultato teorico atteso.

In [152]:
def binary_search_insert(root, value):
    LT = LabelledBinaryTree(None).parent()._element_constructor_
    if not root:
        return LT([], label=value)
    else:
        if value <= root.label():
            fils = binary_search_insert(root[0], value)
            return LT([fils, root[1]], label=root.label())
        else:
            fils = binary_search_insert(root[1], value)
            return LT([root[0], fils], label=root.label())
        
        
def random_perm_of_lenght(n):
    arr = list(range(1, n + 1))
    random.shuffle(arr)
    return Permutation(arr)

        
def permutation_to_bst(perm):
    res = LabelledBinaryTree(None)
    for i in perm:
        res = binary_search_insert(res, i)

    return res

Definiamo ora una procedura che, dato un valore $k$ pari al numero di simulazioni da condurre ovvero il numero di alberi da creare e un valore $n$ ovvero il numero di nodi interni, restituisce il numero totale di foglie presenti interno di ogni albero. Tramite questo valore, come per le altre analisi, potremo facilmente calcolare il numero medio di nodi foglia all'interno degli alberi binari di ricerca creati;

In [158]:
def count_bst_leaves(k, n):
    acc = 0
    for i in tqdm(range(1, k + 1)):
        random_perm = random_perm_of_lenght(n)
        bst = permutation_to_bst(random_perm)
        acc += count_leaves(bst)
    
    return acc

Richiamiamo la procedura con $k=10000$ (ovvero $10000$ alberi) e $n=100$ (ovvero $100$ nodi interni per ogni albero) e lo assegnamo ad una variabile:

In [159]:
k = 10000
n = 100
tot_leaves_bst = count_bst_leaves(k, n)
tot_leaves_bst

100%|██████████| 10000/10000 [01:30<00:00, 110.19it/s]


336853

In [165]:
(tot_leaves_bst / k).n()

33.6853000000000

Come possiamo notare, il risultato ottenuto rispecchia il valore teorico, in quanto si ha che per $n=100$ il numero di nodi foglia dovrebbe risultare circa $33$,  valore ritrovato tramite questa simulazione.

In [157]:
(n / 3).n()

33.3333333333333